In [1]:
from analysis2014.process_2014_data import get_image_vec_list
item_dict = get_image_vec_list()
item_mat = list(item_dict.values())[:50]
num_item_feat = len(item_mat[0])



processed 1000 asins
processed 2000 asins
processed 3000 asins
processed 4000 asins
processed 5000 asins
processed 6000 asins


In [2]:
from analysis2014.process_2014_data import get_mapped_ratings
import pandas as pd
import json
with open("analysis2014/baby_5_core_reviews.json", "r") as pvr_file:
    pv_rev = json.load(pvr_file)
df = pd.DataFrame(pv_rev)

In [3]:
counter = 0
r_id_r_int_mapping = {}
for r_id in pd.unique(df["reviewer_id"]):
    if r_id not in r_id_r_int_mapping:
        r_id_r_int_mapping[r_id] = counter
        counter += 1

In [4]:
counter = 0
p_id_p_int_mapping = {}
for p_id in pd.unique(df["asin"]):
    if p_id not in p_id_p_int_mapping:
        p_id_p_int_mapping[p_id] = counter
        counter += 1

In [5]:
import json
with open("analysis2014/reviews_products.json", "w") as rp:
    json.dump(p_id_p_int_mapping, rp)

with open("analysis2014/reviews_customers.json", "w") as rr:
    json.dump(r_id_r_int_mapping, rr)

In [6]:
df["reviewer_id"] = df["reviewer_id"].map(r_id_r_int_mapping)
df["item_id"] = df["asin"].map(p_id_p_int_mapping)


In [7]:
from cdl_model import CollaborativeDeepLearning
model = CollaborativeDeepLearning(item_mat, [num_item_feat, 512, 64], item_dict)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [8]:
# This should only be done once
model.pretrain(lamda_w=0.001, encoder_noise=0.3, epochs=10)
trained_encoders = model.trained_encoders
trained_decoders = model.trained_decoders

Epoch 1/10
1/1 - 12s - loss: 3.1559
Epoch 2/10
1/1 - 0s - loss: 2.6973
Epoch 3/10
1/1 - 0s - loss: 2.4202
Epoch 4/10
1/1 - 0s - loss: 2.2277
Epoch 5/10
1/1 - 0s - loss: 2.0823
Epoch 6/10
1/1 - 0s - loss: 1.9654
Epoch 7/10
1/1 - 0s - loss: 1.8682
Epoch 8/10
1/1 - 0s - loss: 1.7861
Epoch 9/10
1/1 - 0s - loss: 1.7155
Epoch 10/10
1/1 - 0s - loss: 1.6541
Epoch 1/10
1/1 - 1s - loss: 0.3982
Epoch 2/10
1/1 - 0s - loss: 0.3670
Epoch 3/10
1/1 - 0s - loss: 0.3447
Epoch 4/10
1/1 - 0s - loss: 0.3250
Epoch 5/10
1/1 - 0s - loss: 0.3071
Epoch 6/10
1/1 - 0s - loss: 0.2913
Epoch 7/10
1/1 - 0s - loss: 0.2775
Epoch 8/10
1/1 - 0s - loss: 0.2653
Epoch 9/10
1/1 - 0s - loss: 0.2544
Epoch 10/10
1/1 - 0s - loss: 0.2449


In [9]:
model.trained_encoders = trained_encoders
model.trained_decoders = trained_decoders

In [10]:
train_mat = df.iloc[:500,].sample(frac=0.8,random_state=200) #random state is a seed value
test_mat = df.iloc[:500].drop(train_mat.index)
model_history = model.fineture(train_mat, test_mat, lamda_u=0.01, lamda_v=0.1, lamda_n=0.1, lr=0.01, epochs=3)
testing_rmse = model.getRMSE(test_mat)
print('Testing RMSE = {}'.format(testing_rmse))

Epoch 1/3
7/7 [==============================] - 6s 184ms/step - loss: 517.2087 - dot_layer_loss: 110.5922 - model_2_loss: 0.2081 - val_loss: 478.0798 - val_dot_layer_loss: 75.7116 - val_model_2_loss: 0.1815
Epoch 2/3
7/7 [==============================] - 0s 54ms/step - loss: 481.9117 - dot_layer_loss: 80.2849 - model_2_loss: 0.1786 - val_loss: 470.2220 - val_dot_layer_loss: 70.9306 - val_model_2_los